### How to query the SDMX global registry for DSDs and codelists:
-- Olav ten Bosch, 20210508

In [9]:
import requests                  # for issueing HTTP requests
import json                      # for processing the json data

In [3]:
# Settings for the SDMX global registry API:
# Note: the 'format' querystring parameter is not standard SDMX 2.1 but a convenience from registry.sdmx.org
endpoint = 'https://registry.sdmx.org/ws/public/sdmxapi/rest/'
params = {'format': 'sdmx-json', 'detail': 'full', 'references': 'none'}

In [11]:
# Some utility functions to select the elements useful for validation:
def urn_short(urn):
    return urn.partition("=")[2]
    
def DSD_el(el):
    s = urn_short(el['urn'])
    sep = ' '
    
    representation = el['representation']
    if "textFormat" in representation:
        textformat = representation['textFormat']
        s += sep+textformat['textType']
        if "maxLength" in textformat:
            s += sep+str(textformat['maxLength'])
    elif "representation" in el:
        s += sep+urn_short(representation['representation'])
    else:
        s += sep+el['id']
        
    if "mandatory" in el and el["mandatory"] == True: 
        s += sep+'mandatory'
    return s

In [13]:
# Retrieve a specific codelist:
resource = 'codelist'
agencyID = 'ESTAT'
resourceID = 'CL_ACTIVITY'
version = 'latest'  # 20210508: results in'version '1.7'

r1 = requests.get(f"{endpoint}{resource}/{agencyID}/{resourceID}/{version}", params = params)
print(r1.url, r1.status_code)
#print(r1.text)

# Print codelist name, version and all codes and names:
codelist = json.loads(r1.text)["Codelist"][0]
print(urn_short(codelist['urn']))
for item in codelist['items']:
    print(item['id'], item['names'][0]['value'])


https://registry.sdmx.org/ws/public/sdmxapi/rest/codelist/ESTAT/CL_ACTIVITY/latest?format=sdmx-json&detail=full&references=none 200
ESTAT:CL_ACTIVITY(1.7)
_T Total - All activities
_X Unspecified
_Z Not applicable
A Agriculture, forestry and fishing
A_B Total Primary sector
A01 Crop and animal production, hunting and related service activities
A02 Forestry and logging
A03 Fishing and aquaculture
ATS All activities except activities of households as employers; undifferentiated goods- and services-producing activities of households for own use and activities of extraterritorial organisations and bodies
ATUXFPRV Activities other than financial and insurance
B Mining and quarrying
B05 Mining of coal and lignite
B06 Extraction of crude petroleum and natural gas
B06_09 Extraction of crude petroleum and natural gas; mining support service activities
B07 Mining of metal ores
B08 Other mining and quarrying
B09 Mining support service activities
BTE Industry (except construction)
BTF Industry and

In [14]:
# Retrieve a specific DSD (specific version or the latest):
resource = 'datastructure'
agencyID = 'ESTAT'
resourceID = 'CPI'
version = 'latest' # 20210508: results in version '1.2'

r2 = requests.get(f"{endpoint}{resource}/{agencyID}/{resourceID}/{version}", params = params)
print(r2.url, r1.status_code)
#print(r2.text)

# Print some elements of DSD important for validation:
structure = json.loads(r2.text)["DataStructure"][0]
print(urn_short(structure['urn']))
print('primaryMeasure:')
print(DSD_el(structure['primaryMeasure']))
print('Dimensions and codelists:')
for dimension in structure['dimensionList']['dimensions']:
    print(DSD_el(dimension))
print('Attributes:')
for attribute in structure['attributeList']['attributes']:
    print(DSD_el(attribute))

https://registry.sdmx.org/ws/public/sdmxapi/rest/datastructure/ESTAT/CPI/latest?format=sdmx-json&detail=full&references=none 200
ESTAT:CPI(1.2)
primaryMeasure:
ESTAT:CPI(1.2).OBS_VALUE Double
Dimensions and codelists:
ESTAT:CPI(1.2).FREQ SDMX:CL_FREQ(2.0)
ESTAT:CPI(1.2).SEASONAL_ADJUST ESTAT:CL_SEASON_ADJUST(1.0)
ESTAT:CPI(1.2).REF_AREA IMF:CL_AREA(1.13)
ESTAT:CPI(1.2).IND_TYPE ESTAT:CL_IND_TYPE(1.1)
ESTAT:CPI(1.2).IDX_TYPE ESTAT:CL_IDX_TYPE(1.0)
ESTAT:CPI(1.2).BASE_PER ObservationalTimePeriod
ESTAT:CPI(1.2).ITEM ESTAT:CL_ITEM_PRICE(1.1)
ESTAT:CPI(1.2).COVERAGE_GEO ESTAT:CL_COVERAGE_GEO(1.0)
ESTAT:CPI(1.2).COVERAGE_POP ESTAT:CL_COVERAGE_POP(1.0)
ESTAT:CPI(1.2).UNIT_MEASURE IMF:CL_UNIT(1.14)
ESTAT:CPI(1.2).TRANSFORMATION ESTAT:CL_TRANSFORMATION(1.4)
ESTAT:CPI(1.2).CUST_BREAKDOWN ESTAT:CL_CUST_BREAKDOWN(1.0)
ESTAT:CPI(1.2).TIME_PERIOD ObservationalTimePeriod
Attributes:
ESTAT:CPI(1.2).DISS_ORG IMF:CL_ORGANISATION(1.10)
ESTAT:CPI(1.2).COMPILING_ORG IMF:CL_ORGANISATION(1.10)
ESTAT:CPI(1.2)

In [ ]:
# We dont use this, keep here for reference:
# SDMX formats according to https://github.com/sdmx-twg/sdmx-rest/blob/master/v2_1/ws/rest/docs/rest_cheat_sheet.pdf?raw=true
formats = {
    'SDMX-ML Generic Data': 'application/vnd.sdmx.genericdata+xml;version=2.1',
    'SDMX-ML StructureSpecific Data': 'application/vnd.sdmx.structurespecificdata+xml;version=2.1',
    'SDMX-JSON Data': 'application/vnd.sdmx.data+json;version=1.0.0',
    'SDMX-CSV Data': 'application/vnd.sdmx.data+csv;version=1.0.0',
    'SDMX-ML Structure': 'application/vnd.sdmx.structure+xml;version=2.1',
    'SDMX-JSON Structure': 'application/vnd.sdmx.structure+json;version=1.0.0',
    'SDMX-ML Schemas': 'application/vnd.sdmx.schema+xml;version=2.1',
    'SDMX-ML Generic Metadata': 'application/vnd.sdmx.genericmetadata+xml;version=2.1',
    'SDMX-ML StructureSpecific Meta': 'application/vnd.sdmx.structurespecificmetadata+xml;version=2.1'
}
#Only these three give a 200, the rest a 406 for registry.sdmx.org
headers1 = {'Accept': formats["SDMX-ML StructureSpecific Data"]}
headers2 = {'Accept': formats["SDMX-ML Structure"]}
headers3 = {'Accept': formats["SDMX-ML StructureSpecific Meta"]}
headers = headers3
# It appears that content-negation is not needed for the global registry, we can use a format querystring var
# We keep it here for reference